<a href="https://colab.research.google.com/github/leehon94/Automated-Product-Identification_PJS/blob/master/Keras%20Modelle/Train_multiple_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set up Colab and Drive for Project

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd '/content/gdrive/My Drive/MVP Datensatz/'
train_dir = 'Trainingsdatensatz5'

/content/gdrive/My Drive/MVP Datensatz


In [3]:
from keras import optimizers
from keras import layers
from keras import models
from keras import initializers
from keras import applications
from keras.applications import *

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import backend as K


Using TensorFlow backend.


#Global Variables


In [0]:
img_width = 224
img_height = 224



##Augmentations


In [0]:
featurewise_center = False
samplewise_center = False
featurewise_std_normalization = False
samplewise_std_normalization = False
zca_whitening = False
horizontal_flip = False
vertical_flip = False

brightness_range = None 
preprocessing_function = None

zca_epsilon = 1e-06
rotation_range = 3
width_shift_range = [-50, 0, +50]
height_shift_range = [-50, 0, +50]
shear_range = 0.0
zoom_range = 0.2
channel_shift_range = 0.0
cval = 0.0
interpolation_order = 1

fill_mode = 'nearest'
data_format = 'channels_last'


##Preparing Data

In [0]:
validation_split = 0.4
rescale = 1./255
batch_s = 40


In [0]:
train_datagen = ImageDataGenerator(
                          featurewise_center = featurewise_center,
                          samplewise_center = samplewise_center,
                          featurewise_std_normalization = featurewise_std_normalization,
                          samplewise_std_normalization = samplewise_std_normalization,
                          zca_whitening = zca_whitening,
                          zca_epsilon = zca_epsilon,
                          rotation_range = rotation_range,
                          width_shift_range = width_shift_range,
                          height_shift_range = height_shift_range,
                          brightness_range = brightness_range,
                          shear_range = shear_range,
                          zoom_range = zoom_range,
                          channel_shift_range = channel_shift_range,
                          fill_mode = fill_mode,
                          cval = cval,
                          horizontal_flip = horizontal_flip,
                          vertical_flip = vertical_flip,
                          preprocessing_function = preprocessing_function,
                          data_format = data_format,
                          validation_split = validation_split,
                          interpolation_order = interpolation_order,
                          rescale = rescale,
                          dtype = 'float32'
                          )

In [8]:
train_set = train_datagen.flow_from_directory(
                          train_dir, 
                          target_size = (img_width,img_height),
                          batch_size = batch_s,
                          class_mode = 'categorical',
                          subset = 'training'
                          )


Found 84 images belonging to 5 classes.


In [9]:
validation_set = train_datagen.flow_from_directory(
                          train_dir, 
                          target_size = (img_width,img_height),
                          batch_size = batch_s,
                          class_mode = 'categorical',
                          subset = 'validation'
                          )
       

Found 54 images belonging to 5 classes.



##Create Model


#Select and download different pretrained models

In [10]:
base_models = [
                VGG19(
                weights = 'imagenet',
                include_top = False,
                input_shape = (img_width,img_height,3) 
              ),
                MobileNet(
                    weights='imagenet',
                    include_top=False,
                    input_shape=(img_width,img_height,3)),
              Xception(
                    weights='imagenet',
                    include_top=False,
                    input_shape=(img_width,img_height,3)),
               applications.MobileNetV2(
                   weights='imagenet',
                    include_top=False,
                    input_shape=(img_width,img_height,3)
               ),
               applications.NASNetMobile(weights='imagenet',
                    include_top=False,
                    input_shape=(img_width,img_height,3)),
               applications.VGG16(weights='imagenet',
                    include_top=False,
                    input_shape=(img_width,img_height,3)),
               applications.ResNet152V2(weights='imagenet',
                    include_top=False,
                    input_shape=(img_width,img_height,3)),
               applications.ResNet101V2(weights='imagenet',
                    include_top=False,
                    input_shape=(img_width,img_height,3)),
               applications.DenseNet121(weights='imagenet',
                    include_top=False,
                    input_shape=(img_width,img_height,3))
               
    ]


#specifing optimizer and epochs
opt = optimizers.RMSprop(learning_rate=0.0005)

#evtl. erweiterung für verschiedene optimizer
opts = [optimizers.RMSprop(learning_rate=0.0005),
        optimizers.Adam]

epochs = 1500


29089792/29084464 [==============================] - 1s 0us/step


#Function for initializing and training and saving multiple models

In [0]:
def build_models():
  i = 0
  multiple_models = []
  for base_model in base_models:
      model = models.Sequential()
      model.add(base_model)
      model.add(layers.Flatten())
      model.add(layers.Dropout(0.5))
      model.add(layers.Dense(256, activation = 'relu', kernel_initializer = initializers.RandomNormal(stddev = 0.01), bias_initializer = initializers.Zeros()))
      model.add(layers.Dense(5, activation = 'softmax', kernel_initializer = initializers.RandomNormal(stddev = 0.01), bias_initializer = initializers.Zeros()))
      for layer in base_model.layers:
        layer.trainable = False
      model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
      multiple_models = model.fit_generator(
                    train_set,
                    epochs = epochs,
                    validation_data = validation_set
                    )
      model.save("/content/gdrive/My Drive/Keras Models/" + base_model.name + '.h5')
      
      

  return multiple_models
      
      #model.save("/content/gdrive/My Drive/Keras Models/" + str(base_model.name()))
  


#Function Call

In [0]:
build_models()

Epoch 1/1500
3/3 [==============================] - 31s 10s/step - loss: 1.7538 - acc: 0.2500 - val_loss: 1.9529 - val_acc: 0.2037
Epoch 2/1500
3/3 [==============================] - 2s 696ms/step - loss: 1.7893 - acc: 0.1667 - val_loss: 1.6944 - val_acc: 0.1852
Epoch 3/1500
3/3 [==============================] - 3s 865ms/step - loss: 1.7287 - acc: 0.1786 - val_loss: 1.6381 - val_acc: 0.2407
Epoch 4/1500
3/3 [==============================] - 2s 819ms/step - loss: 1.6312 - acc: 0.2381 - val_loss: 1.4838 - val_acc: 0.1852
Epoch 5/1500
3/3 [==============================] - 3s 834ms/step - loss: 1.7305 - acc: 0.2143 - val_loss: 1.6869 - val_acc: 0.2222
Epoch 6/1500
1/3 [=========>....................] - ETA: 2s - loss: 1.5642 - acc: 0.2250